# Image Classification with CNN of dogs and cats using pytorch

### Import the requiered packages

In [7]:
import io
import requests
from PIL import Image
from torchvision import models, transforms
from torch.autograd import Variable
import torchvision.datasets as datasets
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from random import shuffle
import torch.nn.functional as F
import torch.nn as nn

### transformation to apply to each image so that every image has the same size

In [8]:
transform = transforms.Compose([
    transforms.Scale(130),
    transforms.RandomCrop(128),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
])

### Load train and validation data into datasets.

In [9]:
traindir = "/Users/Dylan/Desktop/dogcat/train/"
validdir = "/Users/Dylan/Desktop/dogcat/smallvalidate/"
train = datasets.ImageFolder(traindir, transform)
valid = datasets.ImageFolder(validdir, transform)

In [10]:
# Apply when in grey sclae
# def convertImage(data):
#     return [data[0][:1], data[1]]

# valid = [convertImage(i) for i in valid]
# train = [convertImage(i) for i in train]

### parameters

In [11]:
EPOCH = 50
BATCH_SIZE = 50
LR = 1e-4

### Prepare train and validation

In [12]:
train_loader = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)

In [13]:
valid_y = [i[1] for i in valid]
valid_y = torch.from_numpy(np.array(valid_y))
valid_x = [i[0] for i in valid]
valid_x = torch.stack(valid_x)
validation_x = Variable(valid_x)

### CNN model

In [14]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 3 * 3, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 2),
            nn.Softmax(),
        )
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [15]:
cnn = CNN()
print(cnn)

CNN (
  (features): Sequential (
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU (inplace)
    (2): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU (inplace)
    (5): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU (inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU (inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU (inplace)
    (12): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
  )
  (classifier): Sequential (
    (0): Dropout (p = 0.5)
    (1): Linear (2304 -> 4096)
    (2): ReLU (inplace)
    (3): Dropout (p = 0.5)
    (4): Linear (4096 -> 4096)
    (5): ReLU (inplace)
    (6): Linear (4096 -> 2)
    (7): Softmax ()
  )
)


### Optimizer and loss function

In [16]:
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
loss_func = nn.CrossEntropyLoss()

In [17]:
for epoch in range(EPOCH):
    for step, (x, y) in enumerate(train_loader):
        b_x = Variable(x)
        b_y = Variable(y)
        output = cnn(b_x)               # cnn output
        loss = loss_func(output, b_y)   # cross entropy loss
        optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()  
        if step % 20 == 0:
            test_output, last_layer = cnn(validation_x)
            pred_y = torch.max(test_output, 1)[1].data.squeeze()
            accuracy = sum(pred_y == valid_y) / float(valid_y.size(0))
            print('Epoch: ', epoch, step,'| train loss: %.4f' % loss.data[0], '| test accuracy: %.2f' % accuracy)